In [ ]:
# Imports

import tensorflow as tf
from tensorflow.keras import datasets, models, layers, applications
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from matplotlib import image
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Setting up an input flow

dataset_path = "../../nature_12K/inaturalist_12K"
train_path = dataset_path + "/train"
test_path = dataset_path + "/val"
max_shape = (256, 256, 3)

data_train = ImageDataGenerator(validation_split=0.1)

data_test = ImageDataGenerator()

train_generator = data_train.flow_from_directory(train_path,
                                                 target_size=(max_shape[0],
                                                              max_shape[1]),
                                                 class_mode='categorical',
                                                 subset='training')

validation_generator = data_train.flow_from_directory(
    train_path,
    target_size=(max_shape[0], max_shape[1]),
    class_mode='categorical',
    subset='validation')

test_generator = data_test.flow_from_directory(test_path,
                                               target_size=(max_shape[0],
                                                            max_shape[1]),
                                               class_mode='categorical')

In [ ]:
# A Function that can generate a CNN modified from a pre-trained model

def finetuneCNN(model_name, retrain):
    base_model = eval(
        'applications' + model_name +
        '(input_shape=max_shape, include_top=False, weights=\'imagenet\')')
    base_model.trainable = True
    non_train = int((1 - retrain) * len(base_model.layers))
    for layer in base_model.layers[:non_train]:
        layer.trainable = False

    inputs = tf.keras.Input(shape=max_shape)
    x = inputs
    x = base_model(x)
    global_average_layer = layers.GlobalAveragePooling2D()
    x = global_average_layer(x)
    prediction_layer = layers.Dense(10)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
# A training function to compile and train the new model

def train(model, optimizer, loss_fn):
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
    model.fit(train_generator,
              epochs=10,
              validation_data=validation_generator,
              callbacks=[WandbCallback()])

In [ ]:
# Setting up hyperparameters for the sweep

sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'model': {
            'values': [
                'InceptionV3', 'InceptionResNetV2', 'ResNet50', 'Xception',
                'NASNetLarge'
            ]
        },
        'retrain': {
            'values': [0.1, 0.15, 0.2]
        }
    }
}


In [ ]:
# Setting up a sweep

sweep_id = wandb.sweep(sweep_config, entity = '0x2e4', project = 'cs6910-a2')

In [ ]:
# Creating a sweep runner function

def run():
    default_config = {'model': 'InceptionResNetV2', 'retrain': 0.1}

    run = wandb.init(project='cs6910-a2', config=default_config)
    config = wandb.config

    model = finetuneCNN(model_name=config.model, retrain=config.retrain)

    optimizer = tf.keras.optimizers.Nadam()
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    train(model, optimizer, loss_fn)

In [ ]:
# Creating a wandb agent

wandb.agent(sweep_id, run)